In [2]:
from unstructured.partition.auto import partition
from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title
import pandas as pd

In [3]:
text_pdf_filename = "samples/layout-parser-paper.pdf"
table_img_pdf_filename = "samples/embedded-images-tables.pdf"
img_pdf_filename = "samples/embedded-images.pdf"
img_filename = "samples/layout-parser-paper-fast.jpg"

prodesa_alameda_pdf_filename = "samples/prodesa/memoria-alameda-del-rio.pdf"

In [3]:
elements = partition(filename=text_pdf_filename, strategy='hi_res')
# chunks_el = chunk_elements(elements, max_characters=1024, overlap=204)

/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
print("Number of elements:", len(elements))
# print("Number of chunks:", len(chunks_el))

# max([len(chunk.text) for chunk in chunks_el])



Number of elements: 178


In [5]:
image_elememts = partition(filename=text_pdf_filename)

In [6]:
pdf_table_elements = partition(filename=table_img_pdf_filename)

In [42]:
img_pdf_elements = partition(filename=img_pdf_filename)

In [4]:
prodesa_alameda_elements = partition(filename=prodesa_alameda_pdf_filename, strategy='hi_res', pdf_infer_table_structure=True)

The pdf_infer_table_structure kwarg is deprecated. Please use skip_infer_table_types instead.
/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

In [24]:
prodesa_alameda_chunks = chunk_by_title(prodesa_alameda_elements, max_characters=15024, overlap=204)

In [12]:
print(prodesa_alameda_chunks[1].text)

% A VIVIENDA A TERRAZAS AREA TOTAL UNIDADES APTO TIPO A 75,35 4,95 80,30 180 APTO TIPO B 65,35 5,02 70,37 171 APTO TIPO C 56,57 5,02 61,59 9 APTO TIPO D 58,54 0,00 58,54 60 APTO TIPO D1 58,69 0,00 58,69 30 APTO TIPO D2 58,89 0,00 58,89 2 APTO TIPO D3 58,59 0,00 58,59 28 APTO TIPO D4 58,77 0,00 58,77 28 APTO TIPO E 49,51 0,00 49,51 2


In [25]:
len(prodesa_alameda_chunks)

7

## Parsing

In [264]:
import requests
import json

url = "http://localhost:8081"
headers = {
    "Content-Type": "application/json"
}

def get_schema(json_nodes, text, schema, outputs=["extraction", "res"], **kwargs):
    data = {
        "flow_id": "LLM_FLW_01HSEZDCTRGPSX11DDWH21K0XX",
        "outputs": outputs,
        "custom_values": {
            "language": "ES_CO",
            "identified_nodes": json_nodes,
            "optional_schema": schema,
            "contextual_text": text,
            **kwargs
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data)).json()["result"]["user_defined_outputs"]
    print(response)

    parsed_value = response[1]["value"]
    # parsed_extraction = response[0]["value"]

    # print(parsed_extraction)
    nodes = parsed_value["entities"]
    relations = parsed_value["relations"]

    nodes_df = pd.DataFrame(nodes)
    relations_df = pd.DataFrame(relations)

    return nodes_df, relations_df

def merge_schemas(df1, df2, join_indices):
    df_merged = pd.merge(df1, df2, on=join_indices, how='outer', suffixes=('_left', '_right'))

    df_merged['props'] = df_merged.apply(
        lambda row: {**(row['props_left'] if isinstance(row['props_left'], dict) else {}),
                     **(row['props_right'] if isinstance(row['props_right'], dict) else {})},
        axis=1
    )

    df_merged['type'] = df_merged.apply(
        lambda row: row['type_left'] if not pd.isna(row['type_left']) else row['type_right'],
        axis=1
    )

    df_final = df_merged[join_indices + ['type', 'props']]

    return df_final

In [253]:
nodes_df, relations_df = None, None
count = 0

for chunk in prodesa_alameda_chunks:
    json_nodes = "None"
    if nodes_df is not None:
        json_nodes = nodes_df.to_json(orient="records")

    _nodes_df, _relations_df = get_schema(json_nodes, chunk.text, "node=Ciudad;node=Macroproyecto;node=Proyecto;node=Tipo_Apto\nedge=HAS")

    if not _nodes_df.empty:
        _nodes_df['props'] = _nodes_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if not _relations_df.empty:
        _relations_df['props'] = _relations_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if nodes_df is None:
        nodes_df = _nodes_df
    else:
        if not _nodes_df.empty:
            nodes_df = merge_schemas(nodes_df, _nodes_df, ['entity_id'])

    if relations_df is None:
        relations_df = _relations_df
    else:
        if not _relations_df.empty:
            relations_df = merge_schemas(relations_df, _relations_df, ['from_entity_id', 'to_entity_id'])

[{'name': 'extraction', 'description': None, 'value': '# Extracted Entities and Relationships\n\n## Entities\n\n1. **Ciudad**\n   - Nombre: Barranquilla\n   - Departamento: Atlántico\n\n2. **Macroproyecto**\n   - Nombre: Alameda del Rio\n   - Ubicación: Barranquilla, Atlántico\n   - Área total: 181 hectáreas\n   - Espacio público: 92 hectáreas\n   - Vivienda, comercio y equipamientos: 89 hectáreas\n   - Parques, alamedas, andenes y ciclorrutas: 50 hectáreas\n   - Modelo: "Ciudad dentro de la Ciudad"\n   - Capacidad: Cerca de 22,000 familias\n\n3. **Proyecto**\n   - Nombre: Pardela\n   - Parte de: Alameda del Rio\n   - Localización: M-R14\n   - Área construida total: 61,358.83 m²\n   - Área de vivienda: 35,817.56 m²\n   - Unidades de vivienda: 510\n   - Estrato socio-económico: 3-4\n   - Torres: 4\n     - Torres A, B, C: 3 torres de 15 pisos, 8 apartamentos por piso\n     - Torre D: 1 torre de 15 pisos, 10 apartamentos por piso\n\n4. **Tipo_Apto**\n   - Tipo: A\n     - Área: 80.0 m²\n  

In [255]:
nodes_df

,entity_id,type,props
0,A,Tipo_Apto,"{'tipo': 'A', 'area': '80.0 m²'}"
1,ALAMEDA_DEL_RIO,Macroproyecto,"{'nombre': 'Alameda del Rio', 'ubicacion': 'Ba..."
2,B,Tipo_Apto,"{'tipo': 'B', 'area': '70.0 m²'}"
3,BARRANQUILLA,Ciudad,"{'nombre': 'Barranquilla', 'departamento': 'At..."
4,C,NaN,"{'tipo': 'C', 'area_vivienda': '56.57 m²', 'ar..."
5,D,Tipo_Apto,"{'tipo': 'D', 'area': '57.0 m²'}"
6,D1,NaN,"{'tipo': 'D1', 'area_vivienda': '58.69 m²', 'a..."
7,D2,NaN,"{'tipo': 'D2', 'area_vivienda': '58.89 m²', 'a..."
8,D3,NaN,"{'tipo': 'D3', 'area_vivienda': '58.59 m²', 'a..."
9,D4,NaN,"{'tipo': 'D4', 'area_vivienda': '58.77 m²', 'a..."


In [263]:
relations_df

,from_entity_id,to_entity_id,type,props
0,ALAMEDA_DEL_RIO,BARRANQUILLA,NaN,{'descripcion': 'El macroproyecto Alameda del ...
1,ALAMEDA_DEL_RIO,PARDELA,HAS,{}
2,BARRANQUILLA,ALAMEDA_DEL_RIO,HAS,{}
3,PARDELA,A,HAS,{}
4,PARDELA,B,HAS,{}
5,PARDELA,C,NaN,{}
6,PARDELA,D,HAS,{}
7,PARDELA,D1,NaN,{}
8,PARDELA,D2,NaN,{}
9,PARDELA,D3,NaN,{}


# CSV Loading

In [258]:
def create_csv(nodes_df, relations_df, filename):
    nodes_df.to_csv(filename + "_nodes.csv", index=False)
    relations_df.to_csv(filename + "_relations.csv", index=False)

In [259]:
create_csv(nodes_df, relations_df, "outputs/prodesa_alameda")

# Load into Neo4J

In [172]:
import dotenv
import os
from neo4j import GraphDatabase

load_status = dotenv.load_dotenv("Neo4j-0c2c4bc6-Created-2024-09-16.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [261]:
import ast

def load_csv_and_create_nodes(file_path):
    with driver.session() as session:
        df = pd.read_csv(file_path)

        for _, row in df.iterrows():
            props = ast.literal_eval(row['props'])

            filtered_props = {k: v for k, v in props.items() if v is not None}
            props_string = ", ".join([f"{k}: ${k}" for k in filtered_props.keys()])

            query = f"""
            MERGE (n:{row['type']} {{id: $entity_id, {props_string}}})
            """

            session.run(query, entity_id=row['entity_id'], **filtered_props)

def load_csv_and_create_relations(file_path):
    with driver.session() as session:
        df = pd.read_csv(file_path)

        for _, row in df.iterrows():
            props = ast.literal_eval(row['props'])

            filtered_props = {k: v for k, v in props.items() if v is not None}
            props_string = ", ".join([f"{k}: ${k}" for k in filtered_props.keys()])

            query = f"""
            MATCH (from:{row['from_entity_type']} {{id: $from_entity_id}})
            MATCH (to:{row['to_entity_type']} {{id: $to_entity_id}})
            MERGE (from)-[r:{row['type']} {{id: $relation_id, {props_string}}}]-(to)
            """

            session.run(query, from_entity_id=row['from_entity_id'], to_entity_id=row['to_entity_id'], **filtered_props)


In [233]:
merges = load_csv_and_create_nodes("outputs/prodesa_alameda_nodes.csv")

print(merges)

/tmp/ipykernel_15573/1156762767.py:4: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


None


In [262]:
load_csv_and_create_relations("outputs/prodesa_alameda_relations.csv")

/tmp/ipykernel_15573/3722110868.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


KeyError: 'from_entity_type'